# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pprint import pprint

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [29]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/train_data_0816.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/test_data_0816.csv")

In [30]:
# 공통 변수 리스트
com_variables_train = [
    'target', 'Model.Suffix', 'Workorder', 'WorkMode Collect Result'
    , 'Dispenser_1', 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

com_variables_test = [
    'target', 'Set ID', 'Model.Suffix', 'Workorder'
    , 'WorkMode Collect Result', 'Dispenser_1'
    , 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

In [31]:
# 공정 이름 필터링 후 공통 변수와 결합
def create_dataset(train_data, test_data, process_name, com_variables_train, com_variables_test):
    # 열 이름 필터링
    Process_Desc_col = train_data.filter(like=process_name).columns
    
    # train 데이터셋 생성
    final_columns_train = list(Process_Desc_col) + com_variables_train
    train_dataset = train_data[final_columns_train]
    
    # test 데이터셋 생성
    final_columns_test = list(Process_Desc_col) + com_variables_test
    test_dataset = test_data[final_columns_test]
    
    return train_dataset, test_dataset

# 공통 변수 정의
## com_variables_train = [...]  -> 이전 코드에서 정의한 변수 사용
## com_variables_test = [...]   -> 이전 코드에서 정의한 변수 사용

# 데이터셋 생성
train_data_dam, test_data_dam = create_dataset(train_data, test_data, '_Dam', com_variables_train, com_variables_test)
train_data_fill1, test_data_fill1 = create_dataset(train_data, test_data, '_Fill1', com_variables_train, com_variables_test)
train_data_fill2, test_data_fill2 = create_dataset(train_data, test_data, '_Fill2', com_variables_train, com_variables_test)
train_data_autoclave, test_data_autoclave = create_dataset(train_data, test_data, '_AutoClave', com_variables_train, com_variables_test)

---

## 모델링

### 모델 정의

In [32]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

# 사용 예시
# train_and_evaluate_model(
#     'lgbm', train_data_fill1,
#     n_estimators=979,
#     num_leaves=1565,
#     max_depth=34,
#     learning_rate=0.04888906225539191,
#     min_child_samples=36,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )


def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

# 사용 예시
# model_dam = fit_all_train_data_function(
#     'lgbm', train_data_dam,
#     n_estimators=2748,
#     num_leaves=657,
#     max_depth=256,
#     learning_rate=0.001043279508273329,
#     min_child_samples=58,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

# # VotingClassifier 사용 예시
# estimators = [
#     ('dam', train_model_Dam),
#     ('autoclave', train_model_AutoClave),
#     ('fill1', train_model_Fill1),
#     ('fill2', train_model_Fill2)
# ]

# # VotingClassifier 학습 및 평가
# voting_clf_hard = train_and_evaluate_voting_classifier(train_data, estimators, voting='hard')
# voting_clf_soft = train_and_evaluate_voting_classifier(train_data, estimators, voting='soft', threshold=0.6)

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

# 사용 예시
# # 예측 확률 리스트 (소프트 보팅용)
# probs = [
#     model_Dam.predict_proba(x_test_dam)[:, 1],
#     model_AutoClave.predict_proba(x_test_autoclave)[:, 1],
#     model_Fill1.predict_proba(x_test_fill1)[:, 1],
#     model_Fill2.predict_proba(x_test_fill2)[:, 1]
# ]

# # 예측 리스트 (하드 보팅용)
# preds = [
#     model_Dam.predict(x_test_dam),
#     model_AutoClave.predict(x_test_autoclave),
#     model_Fill1.predict(x_test_fill1),
#     model_Fill2.predict(x_test_fill2)
# ]

# # 소프트 보팅 결과
# final_predictions_soft = voting(probs, method='soft', threshold=0.3)
# print(sum(final_predictions_soft))

# # 하드 보팅 결과
# final_predictions_hard = voting(preds, method='hard')
# print(sum(final_predictions_hard))



### 모델 학습

Dam 모델

In [33]:
train_model_Dam = train_and_evaluate_model(
    'rf', train_data_dam
    , n_estimators = 1959
    , max_depth = 48
    , min_samples_split = 5
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
    , #verbose=-1
)

rf 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.1927525057825752
---
Confusion Matrix:
[[6930  732]
 [ 315  125]]
---
Accuracy: 0.870772648728709
Precision: 0.14585764294049008
Recall: 0.2840909090909091




AutoClave 모델

In [34]:
train_model_AutoClave = train_and_evaluate_model(
    'xgb', train_data_autoclave
    , n_estimators = 488
    , learning_rate = 0.27456156507923796
    , max_depth = 18
    , alpha = 0.001345329538356762
    , gamma = 0.001271261094255318
    , reg_alpha = 0.8757519133030134
    , reg_lambda = 0.08373579326505055
    , colsample_bytree = 0.8186279659279335
    , subsample = 0.24909941675865316
    , random_state=RANDOM_STATE
    , verbose=-1
)

c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:37:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


xgb 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.19978517722878625
---
Confusion Matrix:
[[7264  398]
 [ 347   93]]
---
Accuracy: 0.9080473957047642
Precision: 0.1894093686354379
Recall: 0.21136363636363636




Fill1 모델

In [35]:
train_model_Fill1 = train_and_evaluate_model(
    'lgbm', train_data_fill1
    , n_estimators=9
    , num_leaves=1565
    , max_depth=34
    , learning_rate=0.04888906225539191 
    , min_child_samples=36
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.23040000000000002
---
Confusion Matrix:
[[7549  113]
 [ 368   72]]
---
Accuracy: 0.9406319427301901
Precision: 0.3891891891891892
Recall: 0.16363636363636364




Fill2 모델

In [36]:
train_model_Fill2 = train_and_evaluate_model(
    'lgbm', train_data_fill2
    , n_estimators=13
    , num_leaves=1813
    , max_depth=105
    , learning_rate=0.0026995515108598628 
    , min_child_samples=37
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.13333333333333333
---
Confusion Matrix:
[[7654    8]
 [ 408   32]]
---
Accuracy: 0.9486546531720563
Precision: 0.8
Recall: 0.07272727272727272




In [37]:
# VotingClassifier 사용 
estimators = [
    ('dam', train_model_Dam),
    ('autoclave', train_model_AutoClave),
    ('fill1', train_model_Fill1),
    ('fill2', train_model_Fill2)
]

# VotingClassifier 학습 및 평가
# voting_clf_hard = voting_function(train_data, estimators, voting='hard')
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.3)

c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:39:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Voting Classifier로 학습한 결과:
F1 Score: 0.2383654937570942
---
Confusion Matrix:
[[7326  336]
 [ 335  105]]
---
Accuracy: 0.9171809429770427
Precision: 0.23809523809523808
Recall: 0.23863636363636365




---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [38]:
model_Dam = fit_all_train_data_function(
    'rf', train_data_dam
    , n_estimators = 1959
    , max_depth = 48
    , min_samples_split = 5
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
    , #verbose=-1
)

model_AutoClave = fit_all_train_data_function(
    'xgb', train_data_autoclave
    , n_estimators = 488
    , learning_rate = 0.27456156507923796
    , max_depth = 18
    , alpha = 0.001345329538356762
    , gamma = 0.001271261094255318
    , reg_alpha = 0.8757519133030134
    , reg_lambda = 0.08373579326505055
    , colsample_bytree = 0.8186279659279335
    , subsample = 0.24909941675865316
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill1 = fit_all_train_data_function(
    'lgbm', train_data_fill1
    , n_estimators=9
    , num_leaves=1565
    , max_depth=34
    , learning_rate=0.04888906225539191 
    , min_child_samples=36
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill2 = fit_all_train_data_function(
    'lgbm', train_data_fill2
    , n_estimators=13
    , num_leaves=1813
    , max_depth=105
    , learning_rate=0.0026995515108598628 
    , min_child_samples=37
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

rf 모델이 train_data_dam 데이터로 학습 완료


c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:41:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


xgb 모델이 train_data_autoclave 데이터로 학습 완료
lgbm 모델이 train_data_fill1 데이터로 학습 완료
lgbm 모델이 train_data_fill2 데이터로 학습 완료


---

Voting(hard)

4개의 모델중 다수의 모델이 판단한 값을 타겟값으로 가져감

In [39]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)

# 각 공정의 예측 확률 계산
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
]

In [40]:
# 예측 리스트 (하드 보팅용)
preds = [
    model_Dam.predict(x_test_dam),
    model_AutoClave.predict(x_test_autoclave),
    model_Fill1.predict(x_test_fill1),
    model_Fill2.predict(x_test_fill2)
]

# 하드 보팅 결과
final_predictions = voting(preds, method='hard')
print(sum(final_predictions))

6


Voting(soft)

4개의 모델의 확률의 합에 대한 평균이 일정 수치(스레스홀드) 값을 넘으면 AbNormal 값을 가짐  

In [41]:
# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1],
    model_AutoClave.predict_proba(x_test_autoclave)[:, 1],
    model_Fill1.predict_proba(x_test_fill1)[:, 1],
    model_Fill2.predict_proba(x_test_fill2)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.3)
print(sum(final_predictions))

842


## 4. 제출하기


### 제출 파일 작성


In [43]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission_voting.csv", index=False)

In [44]:
df_sub['target'].value_counts()

target
Normal      16519
AbNormal      842
Name: count, dtype: int64

In [45]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**


---

## 5. ETC...

모델을 통해 예측해 낸 정답지(csv) 파일의 결과값을 불러와 voting 하는 방식  
이때의 voting은 hard방식만 가능(예측확률은 저장되어있지 않음으로 soft 보팅은 불가능)  

In [46]:
import pandas as pd
import numpy as np

def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

In [47]:
# 제출 파일 경로 리스트
file_paths = [
    "submission1.csv"
    , "submission2.csv"
    , "submission3.csv"
    , "submission4.csv"
    # 파일 추가 가능
    ]

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub["target"] = final_predictions_hard
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub.to_csv("final_submission.csv", index=False)

print("최종 제출 파일이 'final_submission.csv'로 저장되었습니다.")

FileNotFoundError: [Errno 2] No such file or directory: 'submission1.csv'

In [ ]:
df_sub['target'].value_counts()

Normal      17234
AbNormal      127
Name: target, dtype: int64

In [ ]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal
